# Course Timetabling: Estudo de caso para o problema de agendamento de grade horária

## Fase 1: Alocação de professor em disciplina

Objetivo:
- Maximizar a performance acadêmica (EAP, professor habilitado em dar a disciplina) e o número de restrições fracas atendidas
- Alocar professores efetivos e substitutos em disciplinas (obrigatórias, eletivas e de serviço) conforme as regras do instituto da computação da UFRJ

### Modelagem

Conjuntos:

- $P$: Conjunto de professsores $p \in P$ contemplando um professor DUMMY
- $PP \subset P$: Subconjunto de professores permanentes, sem contemplar o professor DUMMY
- $SP \subset P$: Subconjunto de professores substitutos, sem contemplar o professor DUMMY
- $C$: Conjunto de disciplinas (C = Course) ofertadas atrelada a uma turma com o detalhamento da disciplina $c \in C$
- $EC \subset C$: Subconjunto de disciplinas eletivas ($ec \in EC$)
- $RC \subset C$: Subconjunto de disciplinas obrigatórias ($rc \in RC$)
- $Cred_c$: Conjunto de créditos das disciplinas $Cred_c \in C$
- $QC_p \subset C$: Subconjunto de disciplinas nas quais o professor p está habilitado a dar aulas (QC = Qualified Courses). Caso a alocação seja cadastrada manualmente, a disciplina estará dentro deste conjunto $QC$
- $T_c$: Conjunto de horários (T = Time) das disciplinas ($t \in T_c \in C$)
- $D_c$: Conjunto de dias da semana (D = Day) das disciplinas ($d \in D_c \in C$)
- $C_t$: Subconjunto de disciplinas que possuem o horário t ($t \in T_c , C_t \in C$)
- $C_{d}$: Subconjunto de disciplinas que possuem o dia da semana d ($d \in D$)
- $MC_{p,c}$: Conjunto de alocação manual dado um professor e uma disciplina. Entrega a alocação pronta com todos os valores necessários.


<!-- - $T_c$: Subconjunto com todos os horários (T = Time) possíveis para uma disciplina $T_c \in C$ -->
<!-- - $D_c$: Subconjunto com todos os dias da semana possíveis para uma disciplina $D_c \in C$ -->


<!-- - $D_p \subset D$: Preferências de disciplinas de cada professor p -->
<!-- - $Tp \subset H Preferência de turno de cada professor p; -->

#### Variáveis

1) Alocação de professor em disciplina de uma turma 


$$
X_{p,c,d,t} \in (0,1) =

\begin{cases}
1 &  \quad \text{se o professor p for alocado na disciplina c para um dia d e um horário t}\\ 
0 & \quad \text{caso contrário} 
\end{cases}

$$

$$
p \in P, c \in C, d \in D_c, t \in T_c
$$

2. Variável de folga (slack variable) que indica quantos créditos o professor permanente está abaixo do ideal pela coordenação

$$
PNC_{pp} \in \Z \quad pp \in PP
$$ 


#### Coeficientes

Considerando 

$$
    p \in P, c \in C, d \in D_c, t \in T_c
$$

temos que:


1) Professor habilitado em dar disciplina recebe coeficiente não nulo

    1.1) Caso a disciplina seja obrigatória:

    $$

    f_{p,rc,d,t} \in (0,1) = 

    \begin{cases}
    100 &  \quad rc \in QC_p\\ 
    0 & \quad \text{caso contrário}
    \end{cases}
    $$

    $$
        rc \in RC \subset C \wedge p \neq \text{'DUMMY'}
    $$

    1.2) Caso a disciplina seja eletiva:

    $$

    f_{p,ec,d,t} \in (0,1) = 

    \begin{cases}
    1 &  \quad ec \in QC_p\\ 
    0 & \quad \text{caso contrário}
    \end{cases}
    $$

    $$

    c \in C \wedge p \neq \text{'DUMMY'}

    $$

    Assim,

    $$

    f_{p,c,d,t} \in (0,100) = 

    \begin{cases}
    100 &  \quad c \in RC \wedge c \in QC_p\\ 
    1 &  \quad c \in EC \wedge c \in QC_p\\ 
    0 & \quad \text{caso contrário}
    \end{cases}
    $$

    $$

    ec \in EC \subset C \wedge p \neq \text{'DUMMY'}

    $$


2) Professor DUMMY é habilitado em todas as disciplinas. Ele recebe coeficiente 0.0001 pois só deve ser considerado se não existir outro professor apto.

$$

f_{p,c,d,t} = 0.0001 \quad p = \text{'DUMMY'}

$$

3) Fator de penalidade no caso do professor permanente não seja capaz de atingir a quantidade de créditos mínima

$$
    Wf_{pp} = 1000 \quad pp \in PP \subset P
$$



### Função objetivo

Maximizar a performance acadêmica (EAP, professor habilitado em dar a disciplina) 
<!-- e o número de restrições fracas atendidas -->

$$
MAX \sum_{p \in P}\sum_{c \in C}\sum_{d \in D}\sum_{t \in T} f_{p,c,d,t}X_{p,c,d,t} - Wf_{pp} \sum_{pp \in PP} PNC_{pp}
$$

$$
    p \in P, c \in C, d \in D_c, t \in T_c
$$

### Restrições

### Restrições fracas

RF1: Garante que o professor seja alocado com a quantidade de créditos sujerida pela coordenação se possível. Não inviabiliza o modelo caso não seja atingido. 

Considera: 
- $MCP_{pp}$: Valor fixo que representa a quantidade de créditos que um professor permanente deve ministrar. Número de créditos mínimos: 8. Cada professor precisa dar no mínimo 8 créditos horas de aula por semana (cada disciplina geralmente possui 4 créditos).


$$
\sum_{c \in C} Cred_c X_{pp,c,d,t} = MCP_{pp} - PNC_{pp} \quad pp \in PP
$$

> A lógica da função acima descreve que para cada professor fixo, eu passo por todas as disciplinas e, como possuem código/chave única, eu verifico o seu dia (d) e o seu horário (t)

##### Restrições fortes

RH1: Alocações manuais devem ser alocadas no resultado final do modelo

$$
\sum_{ma \in MA_{p,c}} X_{p,c,d,t} = 1 
$$

$$
    p \in ma, c \in ma, d \in ma, t \in ma  \wedge	p \neq \text{'DUMMY'}
$$

RH2: Regime de trabalho (quantidade de horas) - quantidade de créditos máximo para o professor substituto

- $MCS_{ps}$: Número de créditos máximos: 12. Às vezes professores podem dar 12 créditos por semana. Para a equação, temos o professor p fixado e pertencente ao conjunto PS.

$$
\sum_{c \in C} Cred_c X_{ps,c,d,t} \leq MCS_{ps} \quad ps \in PS 
$$

> A lógica da função acima descreve que para cada professor fixo, eu passo por todas as disciplinas e, como possuem código/chave única, eu verifico o seu dia (d) e o seu horário (t)

RH3: Uma disciplina de uma turma, deverá ser ministrada por um único professor

$$
\sum_{p \in P}\sum_{c \in C} X_{p,c,d,t} = 1  \quad d \in D_c, t \in T_c
$$

RH4: Um professor poderá dar no máximo 1 disciplina de uma turma em um mesmo dia e horário (binário OU >= 1)


$$
\sum_{p \in P}\sum_{c \in (C_t \cap C_{d})} X_{p,c,d,t} \leq 1  \quad d \in D_c, t \in T_c  \wedge	p \neq \text{'DUMMY'}
$$

RH5: Um professor não pode lecionar uma disciplina em que ele não esteja apto

$$
\sum_{p \in P}\sum_{c \in (\Omega - QC_c)} X_{p,c,d,t} = 0 \quad d \in D_c, t \in T_c, c \notin (ma \in MA_c) \quad QC_c \subset C
$$

> A alocação sendo manual, não é passada por validação no caso de disciplina apta


